<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

# Introduction

## Goals 
* Give a broad overview of the NeuroGPU code and what it does.
* Show how one can port a NEURON model (.hoc)
* Provide some example of the code and data files for reference.


## Structure
Here are the pertinent files that need to be modified in order to run your own simulations.
1. **runModel.hoc** - This runs the simulation of your model neuron with your experimental stimulations in the Hoc interpreter.
2. **Model File** - This is your main hoc file that represents your neuron and its different parameters/characteristics.
3. **Stimulation Files** - These are the stimulation files (.DAT) that are going to be used to inject current into the neuron.

# Walkthrough

## 1. Stimulation Files
This will be your files that contain your point processes that will simulate an injection of current into your model neuron. They should be in the format of a CSV file.

### Example of Stimulation Files

Run the code cell to show the plot the example data files. If you want to test plot your own stimulation file, just change the 'base' various to wherever your file is located. If it's stored on your computer, change the variable to the filepath of the file. If it's hosted online, change it to the url address.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

path = "https://raw.githubusercontent.com/ttdelgadott/NeuroGPU-Copy/master/UrapNeuron/MainenPy2/Stims/CSV/"
filetype = ".csv"
filenames = ["Ramp025","Ramp05","Ramp1","Ramp15","Ramp2","Step","noiseMean","noiseSTD",\
             "sinsMean10","sinsMean100","sinsMean150","sinsMean200","sinsMean25","sinsMean5",\
             "sinsMean50","sinsSTD10","sinsSTD100","sinsSTD25","sinsSTD5","sinsSTD50"]

for index in range(0,20): 
    %matplotlib inline
    data = pd.read_csv(path+filenames[index]+filetype)
    data = data.T
    a = data.plot(legend=False,figsize=(16, 4))
    b = plt.title(filenames[index])
    c = plt.show()

HTTPError: HTTP Error 404: Not Found

## 2. Hoc Model
This is the file that contains your neuron compartmental model, where each parameter and variable of the neuron is programmed in HOC. The scripts are designed around the [Mainen Pyramidal Neuron (1996)](https://senselab.med.yale.edu/modeldb/showModel.cshtml?model=2488). However, the scripts should be able to work with any hoc model that runs correctly using the Neuron API. 

### Example of hoc model

Run the code cell to hide/show the cell. Toggle option also at the bottom.

In [ ]:
'''
objref sh, axonal, dendritic, dendritic_only
create soma
access soma
  axonOnSoma=1
n_axon_seg = 5
create soma,iseg,hill,myelin[2],node[2]


//RBS code starts here

load_file("./fit.hoc")

//RBS code ends
// --------------------------------------------------------------
// Axon geometry
//
// Similar to Mainen et al (Neuron, 1995)
// --------------------------------------------------------------

proc create_axon() {

  create iseg,hill,myelin[n_axon_seg],node[n_axon_seg]

  soma {
    equiv_diam = sqrt(area(.5)/(4*PI))

    // area = equiv_diam^2*4*PI
  }
  if (numarg()) equiv_diam = $1

  iseg {                // initial segment between hillock + myelin
     L = 15
     nseg = 1
     diam = equiv_diam/10        // see Sloper and Powell 1982, Fig.71
  }

  hill {                
    L = 10
    nseg = 1
    diam(0:1) = 4*iseg.diam:iseg.diam
  }

  // construct myelinated axon with nodes of ranvier

  for i=0,n_axon_seg-1 {
    myelin[i] {         // myelin element
      nseg = 1
      L = 100
      diam = iseg.diam         
    }
    node[i] {           // nodes of Ranvier
      nseg = 1
      L = 1.0           
      diam = iseg.diam*.75       // nodes are thinner than axon
    }
  }

  soma connect hill(0), axonOnSoma       // dend11 -> soma(0) -> soma(1) -> hill
  hill connect iseg(0), 1
  iseg connect myelin[0](0), 1
  myelin[0] connect node[0](0), 1

  for i=0,n_axon_seg-2  { 
      node[i] connect myelin[i+1](0), 1 
      myelin[i+1] connect node[i+1](0), 1
  }
}

// --------------------------------------------------------------
// Spines
// --------------------------------------------------------------

      // Based on the "Folding factor" described in
      // Jack et al (1989), Major et al (1994)
      // note, this assumes active channels are present in spines 
      // at same density as dendrites

spine_dens = 1
      // just using a simple spine density model due to lack of data on some 
      // neuron types.

spine_area = 0.83 // um^2  -- K Harris

proc add_spines() { local a
  forsec $o1 {
    a =0
    for(x) a=a+area(x)

    F = (L*spine_area*spine_dens + a)/a

    L = L * F^(2/3)
    for(x) diam(x) = diam(x) * F^(1/3)
  }
}


proc init_cell() {
  // pas2sive
 forall {
    insert pas2
    Ra = ra 
    cm = c_m 
    g_pas2 = 1/rm
   e_pas2 = v_init
  }
forsec "node" g_pas2 = g_pas2_node


 forsec "dend" {
    insert pas2
    Ra = ra 
    cm = c_m 
    g_pas2 = 1/rm
    e_pas2 = v_init
  }


  // exceptions along the axon
  forsec "myelin" cm = cm_myelin
 forsec "node" g_pas2 = g_pas2_node

  // na+ channels
    forall insert na
  forsec dendritic gbar_na = gna_dend
  forsec "myelin" gbar_na = gna_dend
  hill.gbar_na = gna_node
  iseg.gbar_na = gna_node
  forsec "node" gbar_na = gna_node

  // kv delayed rectifier channels
  iseg { insert kv  gbar_kv = gkv_axon }
  hill { insert kv  gbar_kv = gkv_axon }
  soma { insert kv  gbar_kv = gkv_soma }

  // dendritic channels
  forsec dendritic {
insert km    gbar_km  = gkm_dend
    insert kca   gbar_kca = gkca_dend
    insert ca    gbar_ca = gca_dend
    insert cad
  }

  soma {
    gbar_na = gna_soma
    gbar_km = gkm_soma
    gbar_kca = gkca_soma
    gbar_ca = gca_soma
  }

 
  forall if(ismembrane("k_ion")) ek = Ek
  forall if(ismembrane("na_ion")) {
    ena = Ena
    // seems to be necessary for 3d cells to shift Na kinetics -5 mV
    //vshift_na = -5 //RBS i changed it earlier
  }
  forall if(ismembrane("ca_ion")) {
    eca = Eca
    ion_style("ca_ion",0,1,0,0,0)
    //vshift_ca = 0
  }
  finitialize(v_init)
}


proc load_3dcell() {

// $s1 filename

  aspiny = 0
  forall delete_section()
  xopen($s1)

  access soma

  dendritic = new SectionList()

  // make sure no compartments exceed 50 uM length
  forall {
    diam_save = diam
    n = L/50
    nseg = n + 1
    if (n3d() == 0) diam = diam_save
    dendritic.append()
  }    

  dendritic_only = new SectionList()
  forsec dendritic dendritic_only.append()
  soma  dendritic_only.remove()

  create_axon()
    most = new SectionList()
forall most.append()

  if (!aspiny) add_spines(dendritic_only,spine_dens)
  


}
load_3dcell("./j4a.hoc") 
'''

from IPython.display import HTML
def hide():
    return HTML('''<script>
    code_show=true;
    function code_toggle() {
        if (code_show){
            $('div.cell.code_cell.rendered.selected div.input').hide();
        } else {
            $('div.cell.code_cell.rendered.selected div.input').show();
        }
        code_show = !code_show
    }
    $( document ).ready(code_toggle);
    </script>
    <a href="javascript:code_toggle()">[Toggle Code]</a>''')
hide()

## 3. runModel.py
This python script essentially prepares the model and stimulation files for the extractModel.py script. It also creates a matrix in order to represent the branching structures of the neuron model. This file should be left essentially untouched with exception to the filepaths. Please change the modelFile and stimFile variables to the appropriate paths if you are using your own model/stimulation files. One may also change the outFile and timesFile if they wish.

### Example of runModel.py

Run the code cell to hide/show the cell. Toggle option also at the bottom.

In [ ]:
'''
objref sh, axonal, dendritic, dendritic_only
create soma
access soma
  axonOnSoma=1
n_axon_seg = 5
create soma,iseg,hill,myelin[2],node[2]


//RBS code starts here

load_file("./fit.hoc")

//RBS code ends
// --------------------------------------------------------------
// Axon geometry
//
// Similar to Mainen et al (Neuron, 1995)
// --------------------------------------------------------------

proc create_axon() {

  create iseg,hill,myelin[n_axon_seg],node[n_axon_seg]

  soma {
    equiv_diam = sqrt(area(.5)/(4*PI))

    // area = equiv_diam^2*4*PI
  }
  if (numarg()) equiv_diam = $1

  iseg {                // initial segment between hillock + myelin
     L = 15
     nseg = 1
     diam = equiv_diam/10        // see Sloper and Powell 1982, Fig.71
  }

  hill {                
    L = 10
    nseg = 1
    diam(0:1) = 4*iseg.diam:iseg.diam
  }

  // construct myelinated axon with nodes of ranvier

  for i=0,n_axon_seg-1 {
    myelin[i] {         // myelin element
      nseg = 1
      L = 100
      diam = iseg.diam         
    }
    node[i] {           // nodes of Ranvier
      nseg = 1
      L = 1.0           
      diam = iseg.diam*.75       // nodes are thinner than axon
    }
  }

  soma connect hill(0), axonOnSoma       // dend11 -> soma(0) -> soma(1) -> hill
  hill connect iseg(0), 1
  iseg connect myelin[0](0), 1
  myelin[0] connect node[0](0), 1

  for i=0,n_axon_seg-2  { 
      node[i] connect myelin[i+1](0), 1 
      myelin[i+1] connect node[i+1](0), 1
  }
}

// --------------------------------------------------------------
// Spines
// --------------------------------------------------------------

      // Based on the "Folding factor" described in
      // Jack et al (1989), Major et al (1994)
      // note, this assumes active channels are present in spines 
      // at same density as dendrites

spine_dens = 1
      // just using a simple spine density model due to lack of data on some 
      // neuron types.

spine_area = 0.83 // um^2  -- K Harris

proc add_spines() { local a
  forsec $o1 {
    a =0
    for(x) a=a+area(x)

    F = (L*spine_area*spine_dens + a)/a

    L = L * F^(2/3)
    for(x) diam(x) = diam(x) * F^(1/3)
  }
}


proc init_cell() {
  // pas2sive
 forall {
    insert pas2
    Ra = ra 
    cm = c_m 
    g_pas2 = 1/rm
   e_pas2 = v_init
  }
forsec "node" g_pas2 = g_pas2_node


 forsec "dend" {
    insert pas2
    Ra = ra 
    cm = c_m 
    g_pas2 = 1/rm
    e_pas2 = v_init
  }


  // exceptions along the axon
  forsec "myelin" cm = cm_myelin
 forsec "node" g_pas2 = g_pas2_node

  // na+ channels
    forall insert na
  forsec dendritic gbar_na = gna_dend
  forsec "myelin" gbar_na = gna_dend
  hill.gbar_na = gna_node
  iseg.gbar_na = gna_node
  forsec "node" gbar_na = gna_node

  // kv delayed rectifier channels
  iseg { insert kv  gbar_kv = gkv_axon }
  hill { insert kv  gbar_kv = gkv_axon }
  soma { insert kv  gbar_kv = gkv_soma }

  // dendritic channels
  forsec dendritic {
insert km    gbar_km  = gkm_dend
    insert kca   gbar_kca = gkca_dend
    insert ca    gbar_ca = gca_dend
    insert cad
  }

  soma {
    gbar_na = gna_soma
    gbar_km = gkm_soma
    gbar_kca = gkca_soma
    gbar_ca = gca_soma
  }

 
  forall if(ismembrane("k_ion")) ek = Ek
  forall if(ismembrane("na_ion")) {
    ena = Ena
    // seems to be necessary for 3d cells to shift Na kinetics -5 mV
    //vshift_na = -5 //RBS i changed it earlier
  }
  forall if(ismembrane("ca_ion")) {
    eca = Eca
    ion_style("ca_ion",0,1,0,0,0)
    //vshift_ca = 0
  }
  finitialize(v_init)
}


proc load_3dcell() {

// $s1 filename

  aspiny = 0
  forall delete_section()
  xopen($s1)

  access soma

  dendritic = new SectionList()

  // make sure no compartments exceed 50 uM length
  forall {
    diam_save = diam
    n = L/50
    nseg = n + 1
    if (n3d() == 0) diam = diam_save
    dendritic.append()
  }    

  dendritic_only = new SectionList()
  forsec dendritic dendritic_only.append()
  soma  dendritic_only.remove()

  create_axon()
    most = new SectionList()
forall most.append()

  if (!aspiny) add_spines(dendritic_only,spine_dens)
  


}
load_3dcell("./j4a.hoc") 
'''

from IPython.display import HTML
def hide():
    return HTML('''<script>
    code_show=true;
    function code_toggle() {
        if (code_show){
            $('div.cell.code_cell.rendered.selected div.input').hide();
        } else {
            $('div.cell.code_cell.rendered.selected div.input').show();
        }
        code_show = !code_show
    }
    $( document ).ready(code_toggle);
    </script>
    <a href="javascript:code_toggle()">[Toggle Code]</a>''')
hide()

In [ ]:
%%html
    <script>
    $.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')